This is a notebook to pull down all the information for figure 3 (ncbi hits). The design is to compair between each sample (barcode) instead of each flowcell as they sequenced the same thing. Therefore I just pulled all the ncbi hits for each flowcell for each barcode and merged them together according to the barcode.

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from itertools import product
import argparse
import subprocess
import itertools
from ete3 import NCBITaxa 


In [2]:
ncbi = NCBITaxa()


In [3]:
# use argparse to do this
#BASEDIR = args.BASEDIR
BASEDIR = '/home/yiheng/data/20180524_FAH58271'

# here we define the folder name of the dataframe it created by capturing the folder from the BASDIR
folder_name = os.path.basename(BASEDIR)
#column_name = folder_name.split('_')[-1]

In [4]:
# first check if the analysis folder is there
folder_list = 'analysis  basecalled_data  scripts  tracking  workspace'.split(' ')
for x in range(0,folder_list.count('')):
    folder_list.remove('')
#fix this test
if not set(os.listdir(os.path.abspath(BASEDIR))) == set (folder_list):
    print("Something wrong with basefolder. check it please.")

In [5]:
# get the dataframe there
dataframe = os.path.join(BASEDIR, 'analysis', 'summary_df_%s.tab' % folder_name)
sum_df = pd.read_csv(dataframe, sep='\t')

/home/yiheng/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
sum_df[sum_df.pident_nt > 0].head()

,read_id,passes_filtering,sequence_length_template,mean_qscore_template,barcode_arrangement,barcode_score,kit,variant,pc_survived,qseqid_nt,sseqid_nt,evalue_nt,length_nt,pident_nt,nident_nt,sacc_nt,staxids_nt,scomnames_nt,read_length_pc
0,3de506ef-9b26-4aee-b11e-bf8df5a65299,True,503,10.481,barcode07,84.0513,NB,var1,True,3de506ef-9b26-4aee-b11e-bf8df5a65299,gi|16605664|emb|AL358232.19|,3.000000e-152,430.0,90.70,390.0,AL358232,9606,Homo sapiens,434.0
1,43a2ceb3-ea58-45c4-a9c0-9e845a3ee3f5,True,425,9.72,barcode07,90.8718,NB,var1,True,43a2ceb3-ea58-45c4-a9c0-9e845a3ee3f5,gi|18497210|gb|AC097491.2|,3.000000e-96,352.0,86.65,305.0,AC097491,9606,Homo sapiens,339.0
2,45768783-cf86-4203-a864-57c1be00e671,True,726,8.658,barcode07,90.8718,NB,var1,True,45768783-cf86-4203-a864-57c1be00e671,gi|1871197|gb|U91318.1|HUU91318,7.000000e-165,429.0,92.07,395.0,U91318,9606,Homo sapiens,654.0
4,576ecd5e-1a24-42af-97fd-f0d5fc3a4dde,True,831,9.943,barcode07,91.2308,NB,var1,True,576ecd5e-1a24-42af-97fd-f0d5fc3a4dde,gi|20377037|gb|AC109631.4|,6.000000e-166,460.0,91.09,419.0,AC109631,9606,Homo sapiens,762.0
5,5c0dd0f2-884a-4d7d-857c-62b50825adc2,True,464,8.324,barcode07,79.4359,NB,var1,True,5c0dd0f2-884a-4d7d-857c-62b50825adc2,gi|13235051|emb|AL160237.4|,7.000000e-103,234.0,96.58,226.0,AL160237,9606,Homo sapiens,393.0


In [7]:
# change the mean qscore to float for further filtering
sum_df['mean_qscore_template'] = pd.to_numeric(sum_df['mean_qscore_template'], errors='coerce')

In [8]:
# fills the nan with T/F for easier handling
#sum_df.sseqid_rg.fillna(False, inplace=True)
sum_df.sseqid_nt.fillna(False, inplace=True)
sum_df.mean_qscore_template.fillna(False, inplace=True)
# filter out the rg blast hit
#ntblasthit_reads = sum_df[(sum_df.sseqid_rg == False) & (sum_df.sseqid_nt != False) & (sum_df.passes_filtering == True) & (sum_df.pc_survived == True) & (sum_df.nl_survived == True)]
total_reads = sum_df[(sum_df.sseqid_nt != False) & (sum_df.passes_filtering == True) & (sum_df.pc_survived == True) & (sum_df.mean_qscore_template != False)]
total_reads = total_reads[total_reads['mean_qscore_template'] >= 7]
#ntblasthit_reads = total_reads[total_reads.sseqid_nt != False]
pd.set_option('display.max_columns', None)

#Removing unncessary columns from joint_NCBI_df_taxonomy
#I hard coded to select as it depends on what information you need
for columns in ['passes_filtering', 'barcode_score', 'kit',
       'variant', 'pc_survived', 'qseqid_nt', 'sseqid_nt', 'sacc_nt']:
    del total_reads[columns] 
# keep the common name in to filter the human out

In [9]:
###########This block for barcodes order
# now everything left is what we want to plot out
# arrange different barcodes
# REMEMBER: This need to be manual check:
# now just filter out all the reads that are not going to used for ploting
barcode05_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode05')]
barcode06_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode06')]
barcode07_total = total_reads[total_reads.barcode_arrangement.str.contains('barcode07')]

# now concat them together. but remember that the index has to change to add the column for ncbi taxa
total_reads_filtered_barcodes = pd.concat([barcode05_total, 
                                           barcode06_total, 
                                           barcode07_total], ignore_index=True)

In [10]:
#################### For the analysis of this flowcell's data, everthing above this can be copied directly ################

In [11]:
total_reads_filtered_barcodes.staxids_nt.fillna(False, inplace=True)
ntblasthit_reads_filtered_barcodes = total_reads_filtered_barcodes[(total_reads_filtered_barcodes.staxids_nt != False)]

In [12]:
ntblasthit_reads_barcode07_nohuman = ntblasthit_reads_filtered_barcodes[(ntblasthit_reads_filtered_barcodes.scomnames_nt != 'Homo sapiens') & (ntblasthit_reads_filtered_barcodes.barcode_arrangement == 'barcode07')]
ntblasthit_reads_barcode07_nohuman = ntblasthit_reads_barcode07_nohuman.reset_index(drop=True)

In [13]:
# as the concate before will result the staxid into a string
# so now change the string back to float so it can be recognized by the NCBITaxa
for taxid in ntblasthit_reads_barcode07_nohuman['staxids_nt']:
    if ';' in str(taxid):
        taxid = taxid.split(';')[0]

    else:
        pass
    float(taxid)

In [14]:
#Function to generate taxonomy columns based on NCBITaxa results for NCBI hit dataframe, 
def search_rank_output_name_append_column(df, staxid_column, rank_search):
    """Input df, staxid_column from same df and rank_search (a desired taxonomic rank 
    from each staxid's lineage), outputs taxonomic name corresponding to rank_search or 'Unclassified' if
    unavailable and appends to df row by row"""
    rank_list = []
    for read_index in range(0, len(staxid_column)):
        taxid = ''
        if ';' in str(staxid_column[read_index]):
            taxid = staxid_column[read_index].split(';')[0]
################# BE CAREFUL HERE !!! #######################################
#            df.loc[staxid_column =='71426;73839', 'staxids_nt'] = 73839
        else:
            taxid = staxid_column[read_index]
        
        
        taxid_lineage = ''
        taxid_lineage = ncbi.get_lineage(taxid)
        
        names = ''
        names = ncbi.get_taxid_translator(taxid_lineage)
        
        ranks = ''
        ranks = ncbi.get_rank(taxid_lineage) #Dict
        
        ranks2names = ''
        ranks2names = {ranks[k]:names[k] for k in names.keys() & ranks}
        
        if rank_search in ranks2names.keys():
            rank_list.append(ranks2names[rank_search])#if rank in dict, print name
        else:
            rank_list.append('Unclassified')
    df[rank_search] = rank_list
# NOTE: Appending is always slow, try and find a better way e.g df.apply to a column based on staxids column


In [15]:
#ntblasthit_reads_filtered_barcodes_added_TaxaRank = ntblasthit_reads_filtered_barcodes.copy()
# pretty slow
rank_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for rank in rank_list:
    search_rank_output_name_append_column(ntblasthit_reads_barcode07_nohuman, 
                                          ntblasthit_reads_barcode07_nohuman.staxids_nt, 
                                          rank)

In [16]:
ntblasthit_reads_barcode07_nohuman.head()

,read_id,sequence_length_template,mean_qscore_template,barcode_arrangement,evalue_nt,length_nt,pident_nt,nident_nt,staxids_nt,scomnames_nt,read_length_pc,superkingdom,phylum,class,order,family,genus,species
0,b87a4882-377c-4c97-9ced-86899db77e04,527,8.93,barcode07,1.000000e-115,447.0,85.46,382.0,9598,Pan troglodytes,451.0,Eukaryota,Chordata,Mammalia,Primates,Hominidae,Pan,Pan troglodytes
1,0dbe215c-250a-4443-b550-a77442a75012,432,7.555,barcode07,4.000000e-25,153.0,83.01,127.0,9598,Pan troglodytes,343.0,Eukaryota,Chordata,Mammalia,Primates,Hominidae,Pan,Pan troglodytes
2,b4ac70d2-74b7-41d1-85d0-21d043820632,430,8.16,barcode07,2.000000e-63,209.0,89.95,188.0,9598,Pan troglodytes,299.0,Eukaryota,Chordata,Mammalia,Primates,Hominidae,Pan,Pan troglodytes
3,6535816f-7c3f-47f9-a884-79d02ff46d59,1082,8.571,barcode07,0.000000e+00,579.0,89.98,521.0,318479,Dracunculus medinensis,1006.0,Eukaryota,Nematoda,Chromadorea,Spirurida,Dracunculidae,Dracunculus,Dracunculus medinensis
4,5793d4b0-4a94-4496-a2ad-e8940edad78b,993,8.793,barcode07,0.000000e+00,468.0,93.80,439.0,9598,Pan troglodytes,921.0,Eukaryota,Chordata,Mammalia,Primates,Hominidae,Pan,Pan troglodytes


In [17]:
#total_reads_filtered_barcodes.to_csv(r'/home/yiheng/analysis/WGS/%s_totaltaxa.tab' % column_name, header=column_name, index=None, sep='\t')
ntblasthit_reads_barcode07_nomammal = ntblasthit_reads_barcode07_nohuman[ntblasthit_reads_barcode07_nohuman['class'] != 'Mammalia']
ntblasthit_reads_barcode07_nomammal = ntblasthit_reads_barcode07_nomammal.reset_index(drop = True)

In [18]:
ntblasthit_reads_barcode07_nomammal['similarity_rate'] = ntblasthit_reads_barcode07_nomammal.nident_nt/ntblasthit_reads_barcode07_nomammal.read_length_pc*100

In [ ]:
ntblasthit_reads_barcode07_nomammal.to_csv(r'/home/yiheng/test/analysis/%s_barcode07_nomammal_nttaxa.tab' % folder_name, header='clinical_2', index=None, sep='\t')

In [27]:
ntblasthit_reads_barcode07_nomammal['mean_qscore_template'] = pd.to_numeric(ntblasthit_reads_barcode07_nomammal['mean_qscore_template'], errors='raise')

In [44]:
iterables = [ [55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97],
              [55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97] ]
if not os.path.exists(os.path.join(BASEDIR, 'analysis','blast_pitent_similarity')):
    os.mkdir(os.path.join(BASEDIR, 'analysis','blast_pitent_similarity'))
    
else:
    pass

for t in itertools.product(*iterables):
    clinical_df_fn = os.path.join(BASEDIR, 'analysis','blast_pitent_similarity', 'blast_pitent_similarity_%s_%s.txt' % (t[0], t[1]))
    ntblasthit_reads_barcode07_nomammal[(ntblasthit_reads_barcode07_nomammal.pident_nt > t[0]) & 
                                        (ntblasthit_reads_barcode07_nomammal.similarity_rate > t[1]) &
                                        (ntblasthit_reads_barcode07_nomammal.sequence_length_template > 1000)].to_csv(clinical_df_fn, sep='\t', index=None)

In [47]:
# generate all the species count files. pretty slow. maybe pool all the slow process to run as a script.
blast_pitent_similarity_path = os.path.join(BASEDIR, 'analysis','blast_pitent_similarity')
species_count_path = os.path.join(blast_pitent_similarity_path, 'species_count_for_reads_longer_1kb')
if not os.path.exists(species_count_path):
    os.mkdir(species_count_path)
else:
    pass

In [48]:
for file in os.listdir(blast_pitent_similarity_path):
    file_name = os.path.splitext(file)
    file_path = os.path.join(blast_pitent_similarity_path, '%s') % file
    similarity_rate = file_name[0].split('_')[-2]
    pattern_length = file_name[0].split('_')[-1]
    cut_cmd = 'cut -f 18 %s | sort | uniq -c | sort -nr > %s/species_count_%s_%s.txt' % (file_path, species_count_path, similarity_rate, pattern_length)
    cut_cmd_stderr = subprocess.check_output(cut_cmd, shell=True, stderr=subprocess.STDOUT)
print('species_count_files done.')

species_count_files done.


In [ ]:
clinical_df_fn = os.path.join(BASEDIR, 'analysis','blast_pitent_similarity', 'blast_pitent_similarity_89_94.txt')
clinical_df[(clinical_df.pident_nt > 90) & (clinical_df.similarity_rate > 94)].to_csv(clinical_df_fn, sep='\t', index=None)

In [20]:
df1=pd.DataFrame([[1, 2],[10, 1], [3, 4], [7, 8]], columns=['A', 'B'])
df2=pd.DataFrame([[1, 4],[3, 5]], columns=['A', 'B'])
# now replace the location use the index of each df selected from value by .loc function
# first select the value from df2 column A
for values in df2['A'].unique():
    # then locate the position in df1 and replace it by the VALUE of the position in df2
    df1.loc[df1[df1['A'] == (values)].index, 'B'] = df2.loc[df2[df2['A'] == (values)].index, 'B'].values